In [1]:
!pip install nsetools pandas nsepy PyPortfolioOpt

     |████████████████████████████████| 56 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 6.8 MB 8.5 MB/s eta 0:00:01     |████████████████████████████▋   | 6.1 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 27.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 215 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 921 kB 9.1 MB/s eta 0:00:01
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36083 sha256=ca03aa7e860822155bb7f1a7cf9f87f22496d90039f565a7a31d6c40c6442600
  Stored in directory: /home/jovyan/.cache/pip/wheels/b9/2c/cb/d74199e422a1dbc20297d10cfff9c54d03e0b5c6694d49ccdd
  Created wheel for cvxpy: filename=cvxpy-1.1.12-cp38-cp38-linux_x86_6

In [1]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.read_list(update=True)

In [2]:
nff.fetch_data()

Progress: 0.2% Last ticker: 3MINDIA
Progress: 0.4% Last ticker: ABB
Progress: 0.6% Last ticker: POWERINDIA
Progress: 0.8% Last ticker: ACC
Progress: 1.0% Last ticker: AIAENG


KeyboardInterrupt: 

In [3]:
nff.fetch_metadata()

Progress: 0.2% Last ticker: 3MINDIA
Progress: 0.4% Last ticker: ABB
Progress: 0.6% Last ticker: POWERINDIA
Progress: 0.8% Last ticker: ACC
Progress: 1.0% Last ticker: AIAENG


KeyboardInterrupt: 

In [2]:
from cleaner import Nifty500Cleaner

nfc = Nifty500Cleaner()
nfc.clean_metadata()
nfc.clean_ohlc_data()

In [1]:
from processor import StockProcessor

sp = StockProcessor('3MINDIA', meta_data_loc='data/cleaned/Metadata/nifty500.json')

In [2]:
sp.process_metrics()

In [3]:
sp.stock.metadata

{'Ticker': '3MINDIA',
 'Company Name': '3M India Ltd.',
 'Sector': 'SERVICES',
 'Price': 20710.9,
 'Book Closure End Date': '26-AUG-20',
 'Book Closure Start Date': '21-AUG-20',
 'Ex Date': '19-AUG-20',
 'Purpose of Last Meeting': 'ANNUAL GENERAL MEETING',
 'Record Date': nan,
 'OHLC Data Location': 'data/cleaned/OHLC/Nifty500/3MINDIA.csv',
 'VWAP': 25297.07,
 'Volume': 8051,
 'Previous Close': 25494.35,
 'Turnover': 20366668890000.0,
 'Open': 25499.0,
 'High': 25768.6,
 'Low': 25104.75,
 'Profit Exit': 21746.445,
 'Stop Loss': 19675.355000000003,
 '14MA': 26263.22,
 '50MA': 25952.74,
 '200MA': 21873.22,
 'Daily Return': 0.17,
 'Monthly Return': -6.97,
 'Yearly Return': 32.34,
 'Volatility': 2885.61}

In [1]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", 
                    "data/cleaned/Metadata/nifty500.json", 
                    "data/cleaned/Metadata/proc_nifty500.json")

In [2]:
pp.process_metrics()

File not present: data/cleaned/OHLC/Nifty500/POWERINDIA.csv
File not present: data/cleaned/OHLC/Nifty500/ALOKINDS.csv
File not present: data/cleaned/OHLC/Nifty500/BBTC.csv
File not present: data/cleaned/OHLC/Nifty500/IRB.csv
File not present: data/cleaned/OHLC/Nifty500/M&MFIN.csv
File not present: data/cleaned/OHLC/Nifty500/PVR.csv
File not present: data/cleaned/OHLC/Nifty500/RELAXO.csv
File not present: data/cleaned/OHLC/Nifty500/SBICARD.csv
File not present: data/cleaned/OHLC/Nifty500/SUMICHEM.csv
File not present: data/cleaned/OHLC/Nifty500/SUVENPHAR.csv


In [3]:
import pandas as pd
pd.read_csv('data/cleaned/OHLC/Nifty500/M&M.csv')

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,...,50MA,200MA,200STD,Daily Return,Monthly Return,Yearly Return,Profit Exit,Stop Loss,Bollinger Band Up,Bollinger Band Down
0,2011-06-01,M&M,EQ,672.10,673.0,683.45,672.10,673.95,675.85,678.20,...,NaN,NaN,NaN,NaN,NaN,NaN,709.64,642.06,NaN,NaN
1,2011-06-02,M&M,EQ,675.85,669.0,669.65,657.00,662.00,661.50,662.46,...,NaN,NaN,NaN,-2.12,NaN,NaN,694.58,628.42,NaN,NaN
2,2011-06-03,M&M,EQ,661.50,663.0,673.45,663.00,670.50,669.05,668.81,...,NaN,NaN,NaN,1.14,NaN,NaN,702.50,635.60,NaN,NaN
3,2011-06-06,M&M,EQ,669.05,663.0,665.00,653.25,657.35,657.60,657.91,...,NaN,NaN,NaN,-1.71,NaN,NaN,690.48,624.72,NaN,NaN
4,2011-06-07,M&M,EQ,657.60,655.0,666.70,654.10,664.40,664.25,662.42,...,NaN,NaN,NaN,1.01,NaN,NaN,697.46,631.04,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,2021-04-26,M&M,EQ,777.90,780.0,793.30,775.65,779.80,780.50,782.35,...,835.75,707.89,103.42,0.33,-8.28,129.42,819.52,741.48,825.72,763.49
2452,2021-04-27,M&M,EQ,780.50,781.0,787.95,777.00,778.95,778.40,780.97,...,833.42,709.03,102.94,-0.27,-8.64,124.00,817.32,739.48,823.83,762.08
2453,2021-04-28,M&M,EQ,778.40,786.0,801.60,778.85,780.50,782.10,791.37,...,830.78,710.13,102.53,0.48,-9.46,133.95,821.20,743.00,823.71,761.24
2454,2021-04-29,M&M,EQ,782.10,788.0,792.05,768.60,774.20,774.60,777.24,...,827.98,711.23,102.03,-0.96,-9.49,134.51,813.33,735.87,823.24,759.24


In [1]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", "data/cleaned/Metadata/nifty500.json")
pp.process_close(30)

In [2]:
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

opt1 = EffOptimizer(pp)
# opt2 = HRPOptimizer(pp)
# opt3 = CLAOptimizer(pp)

In [3]:
opt1.optimize_max_sharpe()

/opt/conda/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/pypfopt/risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


In [4]:
opt1.portfolio.statistics

{'Expected Annual Return': 35.0,
 'Annual Volatility': 24.0,
 'Sharpe Ratio': 1.36}

In [5]:
opt1.optimize_min_volatility()

In [6]:
opt1.portfolio.statistics

{'Expected Annual Return': None,
 'Annual Volatility': 11.0,
 'Sharpe Ratio': None}

In [7]:
opt2 = HRPOptimizer(pp)
opt2.optimize()

In [10]:
opt2.portfolio.statistics

{'Expected Annual Return': 54.0,
 'Annual Volatility': 12.0,
 'Sharpe Ratio': 4.19}

In [37]:
temp_df = (pp.close_matrix.pct_change(30).tail(1)*100 > 0)
temp_df

,PIDILITIND,BHARATRAS,SEQUENT,GRSE,BALRAMCHIN,KNRCON,UBL,APLLTD,BANDHANBNK,ITI,...,NESCO,GREAVESCOT,BAYERCROP,SYNGENE,ORIENTELEC,AEGISCHEM,UPL,SANOFI,CANFINHOME,JUBLFOOD
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-30,True,True,True,False,True,False,True,True,False,False,...,False,True,True,True,False,True,False,False,True,False


In [3]:
opt3 = CLAOptimizer(pp)
opt3.optimize_max_sharpe()

KeyboardInterrupt: 

In [ ]:
opt3.portfolio.statistics

In [ ]:
opt3.optimize_min_volatility()
opt3.portfolio.statistics

In [14]:
import pandas as pd
df = pd.read_csv('data/raw/nifty_500_metadata.csv')#.to_dict(orient='records')

In [25]:
some = df[df['symbol']=='M&amp;M'].to_dict(orient='records')

In [26]:
some

[{'Unnamed: 0': 302,
  'pricebandupper': 1005.6,
  'symbol': 'M&amp;M',
  'applicableMargin': 30.19,
  'bcEndDate': '07-AUG-20',
  'totalSellQuantity': 19312.0,
  'adhocMargin': 9.83,
  'companyName': 'Mahindra & Mahindra Limited',
  'marketType': 'N',
  'exDate': '16-JUL-20',
  'bcStartDate': '18-JUL-20',
  'css_status_desc': 'Listed',
  'dayHigh': 918.55,
  'basePrice': 914.2,
  'securityVar': 16.86,
  'pricebandlower': 822.8,
  'sellQuantity5': nan,
  'sellQuantity4': nan,
  'sellQuantity3': nan,
  'cm_adj_high_dt': '08-FEB-21',
  'sellQuantity2': nan,
  'dayLow': 905.0,
  'sellQuantity1': 19312.0,
  'quantityTraded': 2484145.0,
  'pChange': -0.8,
  'totalTradedValue': 22628.33,
  'deliveryToTradedQuantity': 23.39,
  'totalBuyQuantity': nan,
  'averagePrice': 910.91,
  'indexVar': nan,
  'cm_ffm': 86971.7,
  'purpose': 'ANNUAL GENERAL MEETING/DIVIDEND - RS 2.35 PER SHARE',
  'buyPrice2': nan,
  'secDate': '12-Feb-2021 00:00:00',
  'buyPrice1': nan,
  'high52': 952.05,
  'previousClo

In [1]:
from simulate import Backtesting, Agent
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

# bt = Backtesting(proc_ohlc_loc="data/cleaned/OHLC/Nifty500", 
#               proc_metadata_loc="data/cleaned/Metadata/nifty500.json",
#              agents=[Agent(EffOptimizer(), 1000000, optimizer_type='min_vol', 
#                            metadata_loc="data/cleaned/Metadata/nifty500.json")])

# bt = Backtesting(proc_ohlc_loc="data/cleaned/OHLC/Nifty500", 
#               proc_metadata_loc="data/cleaned/Metadata/nifty500.json",
#              agents=[Agent(HRPOptimizer(), 1000000, metadata_loc="data/cleaned/Metadata/nifty500.json")])

bt = Backtesting(proc_ohlc_loc="data/processed/OHLC/Nifty500", 
              proc_metadata_loc="data/processed/Metadata/nifty500.json",
             agents=[Agent(CLAOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.40)])

In [2]:
bt.backtest()

2011-06-01
[]

2011-06-02
[]

2011-06-03
[]

2011-06-06
[]

2011-06-07
[]

2011-06-08
[]

2011-06-09
[]

2011-06-10
[]

2011-06-13
[]

2011-06-14
[]

2011-06-15
[]

2011-06-16
[]

2011-06-17
[]

2011-06-20
[]

2011-06-21
[]

2011-06-22
[]

2011-06-23
[]

2011-06-24
[]

2011-06-27
[]

2011-06-28
[]

2011-06-29
[]

2011-06-30
[]

2011-07-01
[]

2011-07-04
[]

2011-07-05
[]

2011-07-06
[]

2011-07-07
[]

2011-07-08
[]

2011-07-11
[]

2011-07-12
[]

Reallocation day
30
2011-07-13
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLA

2011-07-19
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

2011-07-22
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

2011-07-27
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

2011-07-29
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

...

2011-08-22
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

2011-08-23
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

2011-08-24
['Bought 2 shares of NAUKRI at 1457.0', 'Bought 3 shares of SHRIRAMCIT at 1635.0', 'Bought 2 shares of J&KBANK at 1685.2', 'Bought 3 shares of THERMAX at 1830.0', 'Bought 1 shares of PFIZER at 1458.75', 'Bought 1 shares of SCHAEFFLER at 1178.25', 'Bought 2 shares of SOLARINDS at 1282.8', 'Bought 2 shares of SUDARSCHEM at 1520.0', 'Bought 1 shares of BOSCHLTD at 7058.35', 'Bought 1 shares of GILLETTE at 2135.2', 'Bought 2 shares of TECHM at 1484.5', 'Bought 1 shares of ABBOTINDIA at 1488.1', 'Bought 1 shares of GLAXO at 2330.35', 'Bought 1 shares of GODREJPROP at 811.9', 'Bought 1 shares of WABCOINDIA at 1167.5', 'Bought 1 shares of BAJAJHLDNG at 760.7', 'Bought 1 shares of SANOFI at 2048.15', 'Bought 2 shares of SKFINDIA at 1329.5', 'Bought 2 shares of TORNTPHARM at 1309.4', 'Bought 1 shares of AKZOINDIA at 909.55', 'Bought 2 shares of CANBK at 1019.9', 'Bought 1 shares of ASTRAZEN at 1300.0', 'Bought 1 shares of OIL at 1282.05', 'Bought 1 shares of 3MINDIA at 4151.4', 'Boug

Reallocation day
60


KeyboardInterrupt: 